# Snowflake Notebook Data Engineering

* Author: Jeremiah Hansen
* Last Updated: 2/27/2026

Welcome to the beginning of the Quickstart! Please refer to [the official Snowflake Notebook Data Engineering Quickstart](https://www.snowflake.com/en/developers/guides/data-engineering-with-notebooks/) for all the details including set up steps.

## Step 01 Setup Snowflake

At this point you should have already completed the instructions in the [Setup Snowflake section of the Quickstart](https://www.snowflake.com/en/developers/guides/data-engineering-with-notebooks/#setup-snowflake) to create the Workspace, demo objects, and notebook service.

In [ ]:
%%sql -r dataframe_9
USE ROLE DEMO_ROLE;
USE WAREHOUSE DEMO_WH;
USE SCHEMA DEMO_DB.INTEGRATIONS;

## Step 02 Load Weather

Please follow the instructions in the [Load Weather section of the Quickstart](https://www.snowflake.com/en/developers/guides/data-engineering-with-notebooks/#load-weather) to connect to the Weather Source data share in the Snowflake Marketplace. Here's a recap of the steps:

* Login to Snowsight
* Click on the `Marketplace` -> `Snowflake Marketplace` link in the left navigation bar
* Enter "Weather Source LLC: frostbyte" in the search box and click return
* Click on the "Weather Source LLC: frostbyte" listing tile
* Click the blue "Get" button
    * Expand the "Options" dialog
    * Change the "Database name" to read "FROSTBYTE_WEATHERSOURCE" (all capital letters)
    * Select the "DEMO_ROLE" role to have access to the new database
* Click on the blue "Get" button

In [ ]:
/*---
-- You can also do it via code if you know the account/share details...
SET WEATHERSOURCE_ACCT_NAME = '*** PUT ACCOUNT NAME HERE AS PART OF DEMO SETUP ***';
SET WEATHERSOURCE_SHARE_NAME = '*** PUT ACCOUNT SHARE HERE AS PART OF DEMO SETUP ***';
SET WEATHERSOURCE_SHARE = $WEATHERSOURCE_ACCT_NAME || '.' || $WEATHERSOURCE_SHARE_NAME;

CREATE OR REPLACE DATABASE FROSTBYTE_WEATHERSOURCE
  FROM SHARE IDENTIFIER($WEATHERSOURCE_SHARE);

GRANT IMPORTED PRIVILEGES ON DATABASE FROSTBYTE_WEATHERSOURCE TO ROLE DEMO_ROLE;
---*/

And here's a sample SQL query you can run to view some data from the Weather Source share:

In [ ]:
%%sql -r dataframe_6
-- Let's look at the data - same 3-part naming convention as any other table
SELECT * FROM FROSTBYTE_WEATHERSOURCE.ONPOINT_ID.POSTAL_CODES LIMIT 100;

## Step 03 Load Excel Files

Please follow the instructions in the [Load Excel Files section of the Quickstart](https://www.snowflake.com/en/developers/guides/data-engineering-with-notebooks/#load-excel-files) to open and run the `01_load_excel_files` Notebook. That Notebook will define the pipeline used to load data into the `LOCATION` and `ORDER_DETAIL` tables from the staged Excel files.

## Step 04 Load Daily City Metrics

Please follow the instructions in the [Load Daily City Metrics section of the Quickstart](https://www.snowflake.com/en/developers/guides/data-engineering-with-notebooks/#load-daily-city-metrics) to open and run the `02_load_daily_city_metrics` Notebook. That Notebook will define the pipeline used to create the `DAILY_CITY_METRICS` table.

After running the two notebooks above we can now inspect the logs that were written to [the Snowflake Event table](https://docs.snowflake.com/en/developer-guide/logging-tracing/event-table-setting-up). The query below will help get us started.

In [ ]:
USE ROLE DEMO_ROLE;
USE WAREHOUSE DEMO_WH;
USE SCHEMA DEMO_DB.INTEGRATIONS;

SELECT 
    TIMESTAMP,
    VALUE AS LOG_MESSAGE,
    RESOURCE_ATTRIBUTES:"snow.service.name"::string AS SERVICE_NAME,
    RECORD_ATTRIBUTES:"severity_text"::string AS SEVERITY
FROM DEMO_DB.INTEGRATIONS.DEMO_EVENTS
WHERE RECORD_TYPE = 'LOG'
--  AND RESOURCE_ATTRIBUTES:"snow.service.name" = 'NOTEBOOK_SERVICE'
  AND TIMESTAMP > DATEADD(hour, -1, CURRENT_TIMESTAMP())
ORDER BY TIMESTAMP DESC
LIMIT 100;

## Step 05 Deploy Dev Notebook Project

During this step we will be deploying the dev versions of our two data engineering Notebooks: `01_load_excel_files` and `02_load_daily_city_metrics`. To deploy notebooks to Snowflake we will create a `NOTEBOOK PROJECT` object in our `DEV_SCHEMA`, along with the other resources in our development environment.

Notebook Project Objects (NPOs) are the key to deploying and scheduling notebooks in production with Workspace Notebooks. An NPO is a schema-level object that encapsulates your notebook files and their dependencies, making them ready for scheduled execution.

This code is contained in the `scripts/deploy_notebooks.py` script which is used below and in the deployment CI/CD pipeline later. Please open and review the contents of the script.

In [ ]:
%run scripts/deploy_notebooks.py DEMO_DB DEV_SCHEMA DEMO_PIPELINES_NP ./notebooks

## Step 06 Orchestrate Pipelines

In this step we will create a DAG (or Directed Acyclic Graph) of Tasks using the new [Snowflake Python Management API](https://docs.snowflake.com/en/developer-guide/snowflake-python-api/snowflake-python-overview). The Task DAG API builds upon the Python Management API to provide advanced Task management capabilities. For more details see [Managing Snowflake tasks and task graphs with Python](https://docs.snowflake.com/en/developer-guide/snowflake-python-api/snowflake-python-managing-tasks).

This code is contained in the `scripts/deploy_task_dag.py` script which is used below and in the deployment CI/CD pipeline later. Please open and review the contents of the script.


In [ ]:
%run scripts/deploy_task_dag.py DEMO_DB DEV_SCHEMA DEMO_PIPELINES_NP

In a new tab, switch to the Horizon Catalog, then open and review the new Task DAG that was created. Next manually execute the tasks by opening up the root `DEMO_DAG` task, clicking on the "Graph" tab and then clicking on the "Run Task Graph" play button above the diagram. If not already set, you may have to pick a warehouse to do this.

## Step 07 Deploy to Production

Please follow the instructions in the [Deploy to Production section of the Quickstart](https://www.snowflake.com/en/developers/guides/data-engineering-with-notebooks/#deploy-to-production) to deploy our Notebooks and Task DAG to production.

Here is an outline of the steps:
1. Make a small change to a notebook and commit it to the dev branch
1. Go into GitHub and create a PR and Merge to main branch
1. Review GitHub Actions workflow definition and run results
1. See new production versions of the Notebook Project object and Task DAG in the PROD_SCHEMA
1. Run the production version of the task DAG and see new tables created!

## Step 08 Teardown

Finally, we will tear down our demo environment.

In [ ]:
%%sql -r dataframe_8
USE ROLE ACCOUNTADMIN;

DROP DATABASE DEMO_DB;
DROP WAREHOUSE DEMO_WH;
DROP ROLE DEMO_ROLE;

-- Drop the weather share
DROP DATABASE FROSTBYTE_WEATHERSOURCE;

-- Remove the "dev" branch in your repo